In [1]:
import requests
import pickle
from secrets import *

## Data

In [2]:
base_url = "https://www.downdogapp.com"
session = requests.Session()

## Getting Manifest

Returns the API metadata and the "cred" cookie needed for the rest of the calls

Useful response fields:
* `defaultAudioBalance`
* `defaultShowOverlay`
* `defaultShowCountdown`
* `defaultDisplayEnglishNames`
* `videoQUalities`: [{"label": "Auto", "id": 5}, {"label": "HD", "id": 4}, ..., {"label": "Low", "id": 2}]
* `defaultVideoQualityId`
* `settingsRoots`: List of the available settings:
  * [0]: "id": 16, "selectorType": "PLAYLIST_TYPE", "options": [ {"label": "Alt Beats", ...}, {"id": 2, "label": "Acoustic", ...} ]   (**Attention** "Alt Beats" has no id!)
  * [1]: "id": 15, "selectorType": "SAVASANA_LENGTH", "defaultId": 3, "options": [ {"label": "None"}, {"id": 1, "label": "1 Minute"}, ... {"id": 15, "label": "15 Minutes"} ] (**Attention** "None" has no id!)
  * [2]: "id": 8, "selectorType": "LENGTH", "defaultId": 15, "options": [ {"id": 1, "label": "1",}, ..., {"id": 90, "label": "90"} ]
  * [3]: "affectsLengthOptions": true, "options": [ ] -> Options affecting the session length. (**Attention** No all the voice actors are available for all the practices)
    * [0]: "label": "Full Practice", "children": [ ... ] (**Attention** no id)
    * [1]: "id": 36, "label": "Cardio Flow", "children": [
        * "id": 6, "selectorType": "VOICE_ACTOR", "options": [
            * "id": 113, "label": "Selama", "children": [
                * "id": 18, "selectorType": "VERBOSITY", "defaultId": 1, "options": [ {"id": 1, "label": "Default",}, ... ]
                * "id": 1, "selectorType": "FOCUS_AREA", "defaultId": 0, "options": [ {"id": 13, "label": "Back Strength",}, ... ]
                * "id": 2, "selectorType": "LEVEL", "defaultId": 0, "options": [ 
                    * "id": 1, "label": "Beginner 2", "children": [
                        * "id": 3, "selectorType": "PACE", "defaultId": 4, "options": [ {"id": 2, "label": "Slowest",}, ... ]
                    * ...
    * [2]: "id": 101, "label": "Flexibility Flow", "children": [ ... ]
    * [3]: "id": 3, "label": "Slow Flow",
    * [4]: "id": 8, "label": "Gentle",
    * [5]: "id": 1, "label": "Restorative",
    * [6]: "id": 2, "label": "Quick Flow",
    * [7]: "id": 5, "label": "No Warmup",
    * [8]: "id": 9, "label": "Yin",
    * [9]: "id": 7, "label": "Chair Yoga",
    * [10]: "id": 11, "label": "Ashtanga",
    * [11]: "id": 6, "label": "Sun Salutations",
    * [12]: "id": 20, "label": "Yoga Nidra",
    * [13]: "id": 27, "label": "Hot 26",



In [3]:
res = session.get(f"{base_url}/manifest")
print(res.status_code)
print(res.cookies)
manifest = res.json()

200
<RequestsCookieJar[<Cookie cred=AUDUG98NOXM-AGNBW9SFD7E-9jkm57c8r5io70u85059n64rio for www.downdogapp.com/>]>


In [4]:
settings = manifest["settingRoots"]
playlist_type = settings[0]
savasana_length = settings[1]
length = settings[2]
options_affect_length = settings[3]

In [16]:
options_affect_length["options"][0]["children"][0]["options"][0]["children"][2]["options"][4]

{'id': 3,
 'label': 'Advanced',
 'helpText': 'Give me the hardest poses you have!',
 'showDuringOnboarding': True,
 'children': [{'id': 3,
   'selectorType': 'PACE',
   'affectsLengthOptions': True,
   'defaultId': 4,
   'options': [{'id': 2,
     'label': 'Slowest',
     'requiresPro': True,
     'showDuringOnboarding': True},
    {'label': 'Slow', 'showDuringOnboarding': True},
    {'id': 4, 'label': 'Normal', 'showDuringOnboarding': True},
    {'id': 1, 'label': 'Fast', 'showDuringOnboarding': True},
    {'id': 3,
     'label': 'Fastest',
     'requiresPro': True,
     'showDuringOnboarding': True}]}]}

## Login

In [22]:
data = {
    "email": username,
    "password": password,
}

res = session.post(f"{base_url}/json/login", data=data)
print(res.status_code)
print(res.content)


200
b'{"cred":"A1M2JRLK6DI-AAF5SICAF38-crh4k3cn3dlov5hib53l91ui2c"}'


In [23]:
from datetime import datetime, timezone

expires = None
dumped_cookie = None
for cookie in res.cookies:
    if cookie.name == 'cred':
        expires = cookie.expires
        cred = cookie.value
        dumped_cookie = pickle.dumps(cookie)
print(f"Cred cookie value: {cred} ; expiration: {expires}")

dt = datetime.fromtimestamp(expires, timezone.utc)
datetime.strftime(dt, "%Y%m%dT%H%M%S%Z")

Cred cookie value: A1M2JRLK6DI-AAF5SICAF38-crh4k3cn3dlov5hib53l91ui2c ; expiration: 1650401708


'20220419T205508UTC'

## Generate Session

Takes a `settings` parameter with the following params:
* "0": Practice type
* "1": FOCUS_AREA
* "2": LEVEL
* "3": PACE
* "6": VOICE_ACTOR
* "8": LENGTH
* "15": SAVASANA_LENGTH
* "16": PLAYLIST_TYPE
* "18": VERBOSITY


In [66]:
data = {"settings": '{"16":0,"17":5,"15":3,"8":15,"18":1,"1":0,"3":0,"2":0,"6":8,"0":0}'}

res = session.post(f"{base_url}/json/generate", data=data)
print(res.status_code)
data = res.json()

200


In [72]:
print(f'SequenceId: {data["sequence"]["sequenceId"]}')
print(f'PlaylistId: {data["playlist"]["playlistId"]}')
print(data["sequence"]["poseListItems"][0])

SequenceId: AQHY2AS10FV
PlaylistId: AU5XR3K4CMZ
{'name': "Child's Pose", 'sanskritName': 'Balasana', 'imageUrlSuffix': 'B4crd5ot_1_0_7/180.jpg'}


## Get playback URL

Takes the following video parameters:
* sequenceId
* PlaylistId
* videoOffsetTime: Where to start in the video (if params are changed during the video is played)
* audioBalance: Balance between voice and music (full music is 0, full voice is 1)
* includeCountdown
* 

In [16]:
data = {
	"sequenceId": "A3SR0137ELD",
	"playlistId": "A229CCCKF2L",
	"videoOffsetTime": "0",
	"audioBalance": "0.5",
	"includeCountdown": "true",
	"includeOverlay": "false",
	"includePoseNames": "true",
	"includeSanskritNames": "false",
	"includeClosedCaptions": "false",
	"mirrorVideo": "false",
	"videoQualityId": "5",
	"cellularConnection": "false",
	"chromecast": "false",
	"airplay": "false",
}

res = session.post(f"{base_url}/json/playbackUrl", data=data)
print(res.status_code)
print(res.content)

200
b'{"url":"https://stitched.downdogapp.com/5/74/A3SR0137ELD_ib7eNWA7/master.m3u8"}'


## Video URL

see:
* https://en.wikipedia.org/wiki/M3U
* https://en.wikipedia.org/wiki/HTTP_Live_Streaming

The `master.m3u8` is a playlist having the following format:

```
#EXTM3U
#EXT-X-VERSION:6
#EXT-X-INDEPENDENT-SEGMENTS
#EXT-X-MEDIA:TYPE=AUDIO,GROUP-ID="group_audio",NAME="audio_1",DEFAULT=YES,LANGUAGE="en",URI="1.m3u8"
#EXT-X-STREAM-INF:BANDWIDTH=1234200,AVERAGE-BANDWIDTH=1122000,RESOLUTION=1920x1080,FRAME-RATE=30.000,CODECS="avc1.4d4028,mp4a.40.2",AUDIO="group_audio"
0.m3u8
```

No pause or seek function in video, see:
* https://github.com/xbmc/xbmc/issues/18415
* https://github.com/xbmc/inputstream.adaptive



In [23]:
res = requests.get("https://stitched.downdogapp.com/5/74/A3SR0137ELD_ib7eNWA7/master.m3u8")
print(res.status_code)
print(res.content)

200
b'#EXTM3U\n#EXT-X-VERSION:6\n#EXT-X-INDEPENDENT-SEGMENTS\n#EXT-X-MEDIA:TYPE=AUDIO,GROUP-ID="group_audio",NAME="audio_1",DEFAULT=YES,LANGUAGE="en",URI="1.m3u8"\n#EXT-X-STREAM-INF:BANDWIDTH=1234200,AVERAGE-BANDWIDTH=1122000,RESOLUTION=1920x1080,FRAME-RATE=30.000,CODECS="avc1.4d4028,mp4a.40.2",AUDIO="group_audio"\n0.m3u8'
